# Dataset

The dataset you use is 

Based on this tutorial:
https://docs.aws.amazon.com/en_jp/sagemaker/latest/dg/automatic-model-tuning-ex-data.html

TODO document the dataset

# Session initialisation

In [ ]:
import sys
!{sys.executable} -m pip install "sagemaker>=2.121.0"


In [ ]:
import boto3
import sagemaker
from sagemaker.workflow.pipeline_context import PipelineSession

sagemaker_session = sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

# Parameters

In [ ]:
from time import gmtime, strftime
import time

run_id = f"{strftime('%y%m%d%H%M', gmtime())}"

stage_prefix = "L"
project_prefix = "bank"
variant_prefix = "automl"

In [ ]:
print(f"{run_id=}")

# Data Acquisition

In [ ]:
data_source_uri = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip"

In [ ]:
import os
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

data_folder = os.path.join("./generated/data", stage_prefix, project_prefix, variant_prefix)
os.makedirs(data_folder, exist_ok=True)

raw_data_folder = os.path.join(data_folder, "raw")
os.makedirs(raw_data_folder, exist_ok=True)

response = urlopen(data_source_uri)
source_zip = ZipFile(BytesIO(response.read()))
source_zip.extractall(raw_data_folder) 

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 5)         # Keep the output on one page

In [ ]:
dataset_folder = os.path.join(raw_data_folder, "bank-additional")

In [ ]:
os.listdir(dataset_folder)

In [ ]:
raw_dataset_path = os.path.join(dataset_folder, "bank-additional-full.csv")
df = pd.read_csv(raw_dataset_path, sep=';')

In [ ]:
df.head(10)

# Data Preparation

In [ ]:
# save as comma separated as AutoML requires comma separated CSV

def save_dataset(df, prefix):
    os.makedirs(os.path.join(data_folder, f"{prefix}"), exist_ok=True)
    local_filename = os.path.join(data_folder, f"{prefix}/{prefix}.csv")
    df.to_csv(local_filename, index=False, header=True)
    return local_filename

input_dataset_path = save_dataset(df, "input")

In [ ]:
# uppload
base_uri = f"s3://{default_bucket}/{stage_prefix}/{project_prefix}/{variant_prefix}"

input_data_uri = sagemaker.s3.S3Uploader.upload(
        local_path=input_dataset_path,
        desired_s3_uri=f"{base_uri}/input"
    )

In [ ]:
print(input_data_uri)

In [ ]:
input_df = df = pd.read_csv(input_data_uri)
input_df.head(10)

## Training Parameters

TODO

In [ ]:
max_candidates = 5
validation_fraction = 0.2
total_job_runtime_in_seconds = 1800

label_column = "y"

## AutoML Training

In [ ]:
from sagemaker.automl.automl import AutoML
from sagemaker.automl.automl import AutoMLInput
from sagemaker.workflow.automl_step import AutoMLStep

 
automl = AutoML(
    role=role,
    target_attribute_name=label_column,
    validation_fraction=validation_fraction,
    sagemaker_session=sagemaker_session,
    #total_job_runtime_in_seconds=total_job_runtime_in_seconds, 
    max_candidates = max_candidates,
    mode="ENSEMBLING"
)

input_raw = AutoMLInput(
    inputs=input_data_uri,    
    target_attribute_name=label_column
)

step_args = automl.fit(
    inputs=[input_raw],
    job_name=f"{stage_prefix}-{project_prefix}-{variant_prefix}-J{run_id}"
)


# Inspect the best candidate

In [ ]:
automl.__dict__

In [ ]:
automl.latest_auto_ml_job.__dict__

In [ ]:
best_candidate = automl.best_candidate()

In [ ]:
from pprint import pprint

In [ ]:
pprint(best_candidate)

In [ ]:
print(best_candidate['InferenceContainers'][0]['ModelDataUrl'])

In [ ]:
print(best_candidate['CandidateName'])

In [ ]:
pprint(best_candidate['FinalAutoMLJobObjectiveMetric'])

In [ ]:
pprint(best_candidate['CandidateProperties']['CandidateArtifactLocations'])

In [ ]:
pprint(best_candidate['CandidateProperties']['CandidateMetrics'])

In [ ]:
#pprint(automl.list_candidates())

# Export best candidate

In [ ]:
from sagemaker.s3 import S3Downloader

model_output_folder = os.path.join("./generated/model", stage_prefix, project_prefix, variant_prefix, run_id)
os.makedirs(data_folder, exist_ok=True)


explainability_folder = os.path.join(model_output_folder, "explainability")
os.makedirs(explainability_folder, exist_ok=True)

explainability_uri = best_candidate['CandidateProperties']['CandidateArtifactLocations']['Explainability'] 
S3Downloader.download(s3_uri=explainability_uri,
                      local_path=explainability_folder,
                      sagemaker_session=sagemaker_session)

model_insight_folder = os.path.join(model_output_folder, "model-insight")
os.makedirs(model_insight_folder, exist_ok=True)

model_insights_uri = best_candidate['CandidateProperties']['CandidateArtifactLocations']['ModelInsights'] 
S3Downloader.download(s3_uri=model_insights_uri,
                      local_path=model_insight_folder,
                      sagemaker_session=sagemaker_session)


model_data_folder = os.path.join(model_output_folder, "model-data")
os.makedirs(model_data_folder, exist_ok=True)

model_data_uri = best_candidate['InferenceContainers'][0]['ModelDataUrl'] 
S3Downloader.download(s3_uri=model_data_uri,
                      local_path=model_data_folder,
                      sagemaker_session=sagemaker_session)

esponse = urlopen(data_source_uri)
source_zip = ZipFile(BytesIO(response.read()))
source_zip.extractall(raw_data_folder) 

print(f"{model_output_folder=}")

In [ ]:
import tarfile

tar_filename = os.path.join(model_data_folder, "model.tar.gz")
with tarfile.open(tar_filename, 'r') as archive:
    archive.extractall(model_data_folder)

# TODO

In [ ]:
step_auto_ml_training

## Define a Create Model Step to Create a Model

In order to perform batch transformation using the example model, create a SageMaker model.


In [ ]:
from sagemaker.workflow.model_step import ModelStep

best_auto_ml_model = step_auto_ml_training.get_best_auto_ml_model(
    role, 
    sagemaker_session=pipeline_session
)

step_args_create_model = best_auto_ml_model.create(
    instance_type=instance_type_param
)

step_create_model = ModelStep(
    name="AutoMLCreateModel", 
    step_args=step_args_create_model
) 

## Define a Register Model Step to Create a Model Package


In [ ]:
from sagemaker.model_metrics import ModelMetrics, MetricsSource

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=step_auto_ml_training.properties.BestCandidateProperties.ModelInsightsJsonReportPath,
        content_type="application/json",
    ),
    explainability=MetricsSource(
        s3_uri=step_auto_ml_training.properties.BestCandidateProperties.ExplainabilityJsonReportPath,
        content_type="application/json",
    ),
)

register_args = best_auto_ml_model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics
)

step_register = ModelStep(
    name="AutoMLRegisterModel", 
    step_args=register_args
)